In [1]:
import os, sys

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.retriever.retriever import Retriever
from src.generator.rag_generator import RAGGenerator
from src.pipeline.rag_pipeline import RAGPipeline

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retriever = Retriever(
    index_path=os.path.join(project_root, "models", "faiss_index.bin"),
    metadata_path=os.path.join(project_root, "models", "metadata.pkl"),
    model_name="all-MiniLM-L6-v2",
    k=6,
)

gen = RAGGenerator(model="llama3")
pipeline = RAGPipeline(retriever=retriever, generator=gen, k=6)

Loading embedding model: all-MiniLM-L6-v2 on cpu
Loaded FAISS index from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/faiss_index.bin
Loaded metadata from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/metadata.pkl (n=10710)


In [3]:
history = []

# Turn 1
q1 = "I want a cozy fantasy book with romance."
out1 = pipeline.run(q1, style="detailed", history=history, personality="friendly")

print("ANSWER 1:")
print(out1["answer"])

# Add to history
history.append({"user": q1, "bot": out1["answer"]})


# Turn 2
q2 = "I prefer something with dragons. Any options?"
out2 = pipeline.run(q2, style="detailed", history=history, personality="friendly")

print("\nANSWER 2:")
print(out2["answer"])

# Add to history
history.append({"user": q2, "bot": out2["answer"]})


Device set to use mps:0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ANSWER 1:
What a delightful mood you're in! I'm thrilled to recommend some cozy fantasy books with romance for you. Here are my top picks from your retrieved list:

* **A Caress of Twilight** by Laurell K. Hamilton — This book is a perfect blend of fantasy and romance, with a dash of magic and faeries. As Merry Gentry, the protagonist, navigates the world of faerie courts and politics, she also finds herself entangled in a romantic web. The atmosphere is enchanting, and the story is filled with whimsy and wonder.

* **Words of Silk** by Erin St. Claire/Sandra Brown — This romance novel has a touch of fantasy woven throughout its narrative. With a focus on relationships and intimacy, this book is sure to delight you. Although it's not strictly a fantasy novel, it has elements that will transport you to a world of love and desire.

If nothing else quite fits the bill, I'd like to suggest **Mammoth Book Of Lesbian Short Stories** by Emma Donoghue as a close match. While it's an anthology 

## Multi-turn conversation support:
The generator now accepts history and includes previous messages in the prompt, allowing context-aware follow-up questions.

Personality modes:
Added optional styles: friendly, academic, poetic, sarcastic — controlled through the system prompt.

Improved formatting:
Generator responses now use bullet points, bold titles, and cleaner Markdown for readability.

Safety filters:
Basic checks for empty or harmful queries return a safe message instead of calling the LLM.

#### Example Outputs
Turn 1:

“I want a cozy fantasy book with romance.”
→ Suggestions included A Caress of Twilight, Words of Silk, etc.

Turn 2 (with history):

“I prefer something with dragons.”
→ New, relevant dragon-focused recommendations like A Practical Guide to Dragons, Dragonology, Firedrake.

This confirms that:

-history is used correctly

-style and personality settings work

-recommendations remain grounded in retrieved books